<h1><center>Modeling the relationship between the Incidence Level of Lung and Bronchus and Income Level
</center></h1>



### Import Packages

In [ ]:
%load_ext autoreload
%autoreload 2
import ml_functions as mlf  
import sys
import warnings
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn import preprocessing
from sklearn.cluster import DBSCAN
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
if not sys.warnoptions:
    warnings.simplefilter("ignore")
%matplotlib inline

### Read in Data

In [40]:
lung_df = pd.read_csv('/Users/saptarshighose/Downloads/lung-v03.txt', sep = '\t')

In [41]:
lung_df.head()

,County,"""FIPS""","Age-Adjusted Incidence Rate - cases per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,Average Annual Count,Recent 5-Year Trend in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1
0,"Union County, Florida(6,10)",12125,215.1,183.2,251.1,38,-2.5,-24,25
1,"Owsley County, Kentucky(7,8)",21189,179.7,134.8,236.3,11,4.3,0.9,7.7
2,"McCreary County, Kentucky(7,8)",21147,154.1,130.5,180.9,32,-0.7,-3,1.7
3,"North Slope Borough, Alaska(6,10)",2185,150.8,97.0,220.1,7,-0.2,-36.8,57.8
4,"Powell County, Kentucky(7,8)",21197,140.9,115.2,171.0,22,0.4,-3.3,4.3


### Clean Lung Dataframe

In [42]:
lung_df['County'] = lung_df['County'].str.split('(').str[0]

In [43]:
lung_df.head()

,County,"""FIPS""","Age-Adjusted Incidence Rate - cases per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,Average Annual Count,Recent 5-Year Trend in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1
0,"Union County, Florida",12125,215.1,183.2,251.1,38,-2.5,-24,25
1,"Owsley County, Kentucky",21189,179.7,134.8,236.3,11,4.3,0.9,7.7
2,"McCreary County, Kentucky",21147,154.1,130.5,180.9,32,-0.7,-3,1.7
3,"North Slope Borough, Alaska",2185,150.8,97.0,220.1,7,-0.2,-36.8,57.8
4,"Powell County, Kentucky",21197,140.9,115.2,171.0,22,0.4,-3.3,4.3


In [44]:
lung_df.columns

Index(['County', ' "FIPS"', 'Age-Adjusted Incidence Rate - cases per 100,000',
       'Lower 95% Confidence Interval', 'Upper 95% Confidence Interval',
       'Average Annual Count', 'Recent 5-Year Trend in Incidence Rates',
       'Lower 95% Confidence Interval.1', 'Upper 95% Confidence Interval.1'],
      dtype='object')

In [45]:
county_income_df = pd.read_csv('/Users/saptarshighose/Downloads/census.csv')

In [46]:
county_income_df.head()

,County,FIPS,Value (Dollars),Rank within US (of 3140 counties)
0,United States,0.0,67871,NaN
1,"Stewart County, Georgia",13259.0,22500,"3,137"
2,"McCreary County, Kentucky",21147.0,24063,"3,136"
3,"Holmes County, Mississippi",28051.0,25247,"3,135"
4,"Shannon County, South Dakota",46113.0,26364,"3,134"


### Merge Dataframes

In [47]:
merged_df = pd.merge(lung_df, county_income_df, how='inner', left_on=' "FIPS"', right_on='FIPS')

In [48]:
merged_df.head()

,County_x,"""FIPS""","Age-Adjusted Incidence Rate - cases per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,Average Annual Count,Recent 5-Year Trend in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1,County_y,FIPS,Value (Dollars),Rank within US (of 3140 counties)
0,"Union County, Florida",12125,215.1,183.2,251.1,38,-2.5,-24,25,"Union County, Florida",12125.0,45380,"2,709"
1,"Owsley County, Kentucky",21189,179.7,134.8,236.3,11,4.3,0.9,7.7,"Owsley County, Kentucky",21189.0,43207,"2,836"
2,"McCreary County, Kentucky",21147,154.1,130.5,180.9,32,-0.7,-3,1.7,"McCreary County, Kentucky",21147.0,24063,"3,136"
3,"North Slope Borough, Alaska",2185,150.8,97.0,220.1,7,-0.2,-36.8,57.8,"North Slope Borough, Alaska",2185.0,77330,303
4,"Powell County, Kentucky",21197,140.9,115.2,171.0,22,0.4,-3.3,4.3,"Powell County, Kentucky",21197.0,48545,"2,462"


In [49]:
merged_df.columns

Index(['County_x', ' "FIPS"',
       'Age-Adjusted Incidence Rate - cases per 100,000',
       'Lower 95% Confidence Interval', 'Upper 95% Confidence Interval',
       'Average Annual Count', 'Recent 5-Year Trend in Incidence Rates',
       'Lower 95% Confidence Interval.1', 'Upper 95% Confidence Interval.1',
       'County_y', 'FIPS', 'Value (Dollars)',
       'Rank within US (of 3140 counties)'],
      dtype='object')

In [50]:
merged_df = merged_df[[
       'Age-Adjusted Incidence Rate - cases per 100,000',
       'County_y', 'Value (Dollars)']]

### Dummify Categorical Variables

In [51]:
county_dummies = pd.get_dummies(merged_df['County_y'])

In [52]:
county_dummies.head()

,"Abbeville County, South Carolina","Acadia Parish, Louisiana","Accomack County, Virginia","Ada County, Idaho","Adair County, Iowa","Adair County, Kentucky","Adair County, Missouri","Adair County, Oklahoma","Adams County, Colorado","Adams County, Idaho","Adams County, Illinois","Adams County, Indiana","Adams County, Iowa","Adams County, Mississippi","Adams County, Nebraska","Adams County, Ohio","Adams County, Pennsylvania","Adams County, Washington","Adams County, Wisconsin","Addison County, Vermont","Aiken County, South Carolina","Alachua County, Florida","Alamance County, North Carolina","Alameda County, California","Alamosa County, Colorado","Albany County, New York","Albany County, Wyoming","Albemarle County, Virginia","Alcona County, Michigan","Alcorn County, Mississippi","Alexander County, Illinois","Alexander County, North Carolina","Alexandria City, Virginia","Alfalfa County, Oklahoma","Alger County, Michigan","Allamakee County, Iowa","Allegan County, Michigan","Allegany County, Maryland","Allegany County, New York","Alleghany County and Clifton Forge City, Virginia","Alleghany County, North Carolina","Allegheny County, Pennsylvania","Allen County, Indiana","Allen County, Kentucky","Allen County, Ohio","Allen Parish, Louisiana","Allendale County, South Carolina","Alpena County, Michigan","Amador County, California","Amelia County, Virginia","Amherst County, Virginia","Amite County, Mississippi","Anchorage Borough, Alaska","Anderson County, Kentucky","Anderson County, South Carolina","Anderson County, Tennessee","Anderson County, Texas","Andrew County, Missouri","Andrews County, Texas","Androscoggin County, Maine","Angelina County, Texas","Anne Arundel County, Maryland","Anson County, North Carolina","Antelope County, Nebraska","Antrim County, Michigan","Apache County, Arizona","Appanoose County, Iowa","Appling County, Georgia","Appomattox County, Virginia","Aransas County, Texas","Arapahoe County, Colorado","Archer County, Texas","Archuleta County, Colorado","Arenac County, Michigan","Arkansas County, Arkansas","Arlington County, Virginia","Armstrong County, Pennsylvania","Aroostook County, Maine","Ascension Parish, Louisiana","Ashe County, North Carolina","Ashland County, Ohio","Ashland County, Wisconsin","Ashley County, Arkansas","Ashtabula County, Ohio","Asotin County, Washington","Assumption Parish, Louisiana","Atascosa County, Texas","Atchison County, Missouri","Athens County, Ohio","Atkinson County, Georgia","Atlantic County, New Jersey","Atoka County, Oklahoma","Attala County, Mississippi","Audrain County, Missouri","Audubon County, Iowa","Auglaize County, Ohio","Augusta County, Virginia","Austin County, Texas","Autauga County, Alabama","Avery County, North Carolina","Avoyelles Parish, Louisiana","Baca County, Colorado","Bacon County, Georgia","Baker County, Florida","Baker County, Georgia","Baker County, Oregon","Baldwin County, Alabama","Baldwin County, Georgia","Ballard County, Kentucky","Baltimore City, Maryland","Baltimore County, Maryland","Bamberg County, South Carolina","Bandera County, Texas","Banks County, Georgia","Bannock County, Idaho","Baraga County, Michigan","Barbour County, Alabama","Barbour County, West Virginia","Barnes County, North Dakota","Barnstable County, Massachusetts","Barnwell County, South Carolina","Barren County, Kentucky","Barron County, Wisconsin","Barrow County, Georgia","Barry County, Michigan","Barry County, Missouri","Bartholomew County, Indiana","Barton County, Missouri","Bartow County, Georgia","Bastrop County, Texas","Bates County, Missouri","Bath County, Kentucky","Bath County, Virginia","Baxter County, Arkansas","Bay County, Florida","Bay County, Michigan","Bayfield County, Wisconsin","Beadle County, South Dakota","Beaufort County, North Carolina","Beaufort County, South Carolina","Beauregard Parish, Louisiana","Beaver County, Pennsylvania","Beaverhead County, Montana","Beckham County, Oklahoma","Bedford City and County, Virginia","Bedford County, Pennsylvania","Bedford 

In [53]:
county_dummies['Abbeville County, South Carolina'].unique()

array([0, 1], dtype=uint64)

In [54]:
county_dummies.shape

(2708, 2708)

### Join County Dummies with Merged Dataframe

In [55]:
merged_df = merged_df.join(county_dummies)

In [56]:
merged_df['Abbeville County, South Carolina'].unique()

array([0, 1], dtype=uint64)

In [57]:
merged_df

,"Age-Adjusted Incidence Rate - cases per 100,000",County_y,Value (Dollars),"Abbeville County, South Carolina","Acadia Parish, Louisiana","Accomack County, Virginia","Ada County, Idaho","Adair County, Iowa","Adair County, Kentucky","Adair County, Missouri","Adair County, Oklahoma","Adams County, Colorado","Adams County, Idaho","Adams County, Illinois","Adams County, Indiana","Adams County, Iowa","Adams County, Mississippi","Adams County, Nebraska","Adams County, Ohio","Adams County, Pennsylvania","Adams County, Washington","Adams County, Wisconsin","Addison County, Vermont","Aiken County, South Carolina","Alachua County, Florida","Alamance County, North Carolina","Alameda County, California","Alamosa County, Colorado","Albany County, New York","Albany County, Wyoming","Albemarle County, Virginia","Alcona County, Michigan","Alcorn County, Mississippi","Alexander County, Illinois","Alexander County, North Carolina","Alexandria City, Virginia","Alfalfa County, Oklahoma","Alger County, Michigan","Allamakee County, Iowa","Allegan County, Michigan","Allegany County, Maryland","Allegany County, New York","Alleghany County and Clifton Forge City, Virginia","Alleghany County, North Carolina","Allegheny County, Pennsylvania","Allen County, Indiana","Allen County, Kentucky","Allen County, Ohio","Allen Parish, Louisiana","Allendale County, South Carolina","Alpena County, Michigan","Amador County, California","Amelia County, Virginia","Amherst County, Virginia","Amite County, Mississippi","Anchorage Borough, Alaska","Anderson County, Kentucky","Anderson County, South Carolina","Anderson County, Tennessee","Anderson County, Texas","Andrew County, Missouri","Andrews County, Texas","Androscoggin County, Maine","Angelina County, Texas","Anne Arundel County, Maryland","Anson County, North Carolina","Antelope County, Nebraska","Antrim County, Michigan","Apache County, Arizona","Appanoose County, Iowa","Appling County, Georgia","Appomattox County, Virginia","Aransas County, Texas","Arapahoe County, Colorado","Archer County, Texas","Archuleta County, Colorado","Arenac County, Michigan","Arkansas County, Arkansas","Arlington County, Virginia","Armstrong County, Pennsylvania","Aroostook County, Maine","Ascension Parish, Louisiana","Ashe County, North Carolina","Ashland County, Ohio","Ashland County, Wisconsin","Ashley County, Arkansas","Ashtabula County, Ohio","Asotin County, Washington","Assumption Parish, Louisiana","Atascosa County, Texas","Atchison County, Missouri","Athens County, Ohio","Atkinson County, Georgia","Atlantic County, New Jersey","Atoka County, Oklahoma","Attala County, Mississippi","Audrain County, Missouri","Audubon County, Iowa","Auglaize County, Ohio","Augusta County, Virginia","Austin County, Texas","Autauga County, Alabama","Avery County, North Carolina","Avoyelles Parish, Louisiana","Baca County, Colorado","Bacon County, Georgia","Baker County, Florida","Baker County, Georgia","Baker County, Oregon","Baldwin County, Alabama","Baldwin County, Georgia","Ballard County, Kentucky","Baltimore City, Maryland","Baltimore County, Maryland","Bamberg County, South Carolina","Bandera County, Texas","Banks County, Georgia","Bannock County, Idaho","Baraga County, Michigan","Barbour County, Alabama","Barbour County, West Virginia","Barnes County, North Dakota","Barnstable County, Massachusetts","Barnwell County, South Carolina","Barren County, Kentucky","Barron County, Wisconsin","Barrow County, Georgia","Barry County, Michigan","Barry County, Missouri","Bartholomew County, Indiana","Barton County, Missouri","Bartow County, Georgia","Bastrop County, Texas","Bates County, Missouri","Bath County, Kentucky","Bath County, Virginia","Baxter County, Arkansas","Bay County, Florida","Bay County, Michigan","Bayfield County, Wisconsin","Beadle County, South Dakota","Beaufort County, North Carolina","Beaufort County, South Carolina","Beauregard Parish, Louisiana","Beaver County, Pennsylvania","Beaverhead County, Montana","Beckham County, Oklahoma","

In [58]:
merged_df = merged_df.drop(['County_y'], axis=1)

In [59]:
merged_df.head()

,"Age-Adjusted Incidence Rate - cases per 100,000",Value (Dollars),"Abbeville County, South Carolina","Acadia Parish, Louisiana","Accomack County, Virginia","Ada County, Idaho","Adair County, Iowa","Adair County, Kentucky","Adair County, Missouri","Adair County, Oklahoma","Adams County, Colorado","Adams County, Idaho","Adams County, Illinois","Adams County, Indiana","Adams County, Iowa","Adams County, Mississippi","Adams County, Nebraska","Adams County, Ohio","Adams County, Pennsylvania","Adams County, Washington","Adams County, Wisconsin","Addison County, Vermont","Aiken County, South Carolina","Alachua County, Florida","Alamance County, North Carolina","Alameda County, California","Alamosa County, Colorado","Albany County, New York","Albany County, Wyoming","Albemarle County, Virginia","Alcona County, Michigan","Alcorn County, Mississippi","Alexander County, Illinois","Alexander County, North Carolina","Alexandria City, Virginia","Alfalfa County, Oklahoma","Alger County, Michigan","Allamakee County, Iowa","Allegan County, Michigan","Allegany County, Maryland","Allegany County, New York","Alleghany County and Clifton Forge City, Virginia","Alleghany County, North Carolina","Allegheny County, Pennsylvania","Allen County, Indiana","Allen County, Kentucky","Allen County, Ohio","Allen Parish, Louisiana","Allendale County, South Carolina","Alpena County, Michigan","Amador County, California","Amelia County, Virginia","Amherst County, Virginia","Amite County, Mississippi","Anchorage Borough, Alaska","Anderson County, Kentucky","Anderson County, South Carolina","Anderson County, Tennessee","Anderson County, Texas","Andrew County, Missouri","Andrews County, Texas","Androscoggin County, Maine","Angelina County, Texas","Anne Arundel County, Maryland","Anson County, North Carolina","Antelope County, Nebraska","Antrim County, Michigan","Apache County, Arizona","Appanoose County, Iowa","Appling County, Georgia","Appomattox County, Virginia","Aransas County, Texas","Arapahoe County, Colorado","Archer County, Texas","Archuleta County, Colorado","Arenac County, Michigan","Arkansas County, Arkansas","Arlington County, Virginia","Armstrong County, Pennsylvania","Aroostook County, Maine","Ascension Parish, Louisiana","Ashe County, North Carolina","Ashland County, Ohio","Ashland County, Wisconsin","Ashley County, Arkansas","Ashtabula County, Ohio","Asotin County, Washington","Assumption Parish, Louisiana","Atascosa County, Texas","Atchison County, Missouri","Athens County, Ohio","Atkinson County, Georgia","Atlantic County, New Jersey","Atoka County, Oklahoma","Attala County, Mississippi","Audrain County, Missouri","Audubon County, Iowa","Auglaize County, Ohio","Augusta County, Virginia","Austin County, Texas","Autauga County, Alabama","Avery County, North Carolina","Avoyelles Parish, Louisiana","Baca County, Colorado","Bacon County, Georgia","Baker County, Florida","Baker County, Georgia","Baker County, Oregon","Baldwin County, Alabama","Baldwin County, Georgia","Ballard County, Kentucky","Baltimore City, Maryland","Baltimore County, Maryland","Bamberg County, South Carolina","Bandera County, Texas","Banks County, Georgia","Bannock County, Idaho","Baraga County, Michigan","Barbour County, Alabama","Barbour County, West Virginia","Barnes County, North Dakota","Barnstable County, Massachusetts","Barnwell County, South Carolina","Barren County, Kentucky","Barron County, Wisconsin","Barrow County, Georgia","Barry County, Michigan","Barry County, Missouri","Bartholomew County, Indiana","Barton County, Missouri","Bartow County, Georgia","Bastrop County, Texas","Bates County, Missouri","Bath County, Kentucky","Bath County, Virginia","Baxter County, Arkansas","Bay County, Florida","Bay County, Michigan","Bayfield County, Wisconsin","Beadle County, South Dakota","Beaufort County, North Carolina","Beaufort County, South Carolina","Beauregard Parish, Louisiana","Beaver County, Pennsylvania","Beaverhead County, Montana","Beckham County, Oklahoma","Bedford C

In [60]:
merged_df.to_csv('merged_df_test2.csv')

In [2]:
merged_df = pd.read_csv('merged_df_test2.csv')

In [3]:
merged_df.shape

(2708, 2708)

### Isolate Label

In [4]:
label = merged_df.loc[:, merged_df.columns == 'Age-Adjusted Incidence Rate - cases per 100,000']  #Label

In [5]:
label.head()

,"Age-Adjusted Incidence Rate - cases per 100,000"
0,76.5
1,67.2
2,37.4
3,38.9
4,37.5


### Isolate Features

In [6]:
features = merged_df.loc[:, merged_df.columns != 'Age-Adjusted Incidence Rate - cases per 100,000'] 

In [7]:
features.head()

,Value (Dollars),"Abbeville County, South Carolina","Acadia Parish, Louisiana","Accomack County, Virginia","Ada County, Idaho","Adair County, Iowa","Adair County, Kentucky","Adair County, Missouri","Adair County, Oklahoma","Adams County, Colorado","Adams County, Idaho","Adams County, Illinois","Adams County, Indiana","Adams County, Iowa","Adams County, Mississippi","Adams County, Nebraska","Adams County, Ohio","Adams County, Pennsylvania","Adams County, Washington","Adams County, Wisconsin","Addison County, Vermont","Aiken County, South Carolina","Alachua County, Florida","Alamance County, North Carolina","Alameda County, California","Alamosa County, Colorado","Albany County, New York","Albany County, Wyoming","Albemarle County, Virginia","Alcona County, Michigan","Alcorn County, Mississippi","Alexander County, Illinois","Alexander County, North Carolina","Alexandria City, Virginia","Alfalfa County, Oklahoma","Alger County, Michigan","Allamakee County, Iowa","Allegan County, Michigan","Allegany County, Maryland","Allegany County, New York","Alleghany County and Clifton Forge City, Virginia","Alleghany County, North Carolina","Allegheny County, Pennsylvania","Allen County, Indiana","Allen County, Kentucky","Allen County, Ohio","Allen Parish, Louisiana","Allendale County, South Carolina","Alpena County, Michigan","Amador County, California","Amelia County, Virginia","Amherst County, Virginia","Amite County, Mississippi","Anchorage Borough, Alaska","Anderson County, Kentucky","Anderson County, South Carolina","Anderson County, Tennessee","Anderson County, Texas","Andrew County, Missouri","Andrews County, Texas","Androscoggin County, Maine","Angelina County, Texas","Anne Arundel County, Maryland","Anson County, North Carolina","Antelope County, Nebraska","Antrim County, Michigan","Apache County, Arizona","Appanoose County, Iowa","Appling County, Georgia","Appomattox County, Virginia","Aransas County, Texas","Arapahoe County, Colorado","Archer County, Texas","Archuleta County, Colorado","Arenac County, Michigan","Arkansas County, Arkansas","Arlington County, Virginia","Armstrong County, Pennsylvania","Aroostook County, Maine","Ascension Parish, Louisiana","Ashe County, North Carolina","Ashland County, Ohio","Ashland County, Wisconsin","Ashley County, Arkansas","Ashtabula County, Ohio","Asotin County, Washington","Assumption Parish, Louisiana","Atascosa County, Texas","Atchison County, Missouri","Athens County, Ohio","Atkinson County, Georgia","Atlantic County, New Jersey","Atoka County, Oklahoma","Attala County, Mississippi","Audrain County, Missouri","Audubon County, Iowa","Auglaize County, Ohio","Augusta County, Virginia","Austin County, Texas","Autauga County, Alabama","Avery County, North Carolina","Avoyelles Parish, Louisiana","Baca County, Colorado","Bacon County, Georgia","Baker County, Florida","Baker County, Georgia","Baker County, Oregon","Baldwin County, Alabama","Baldwin County, Georgia","Ballard County, Kentucky","Baltimore City, Maryland","Baltimore County, Maryland","Bamberg County, South Carolina","Bandera County, Texas","Banks County, Georgia","Bannock County, Idaho","Baraga County, Michigan","Barbour County, Alabama","Barbour County, West Virginia","Barnes County, North Dakota","Barnstable County, Massachusetts","Barnwell County, South Carolina","Barren County, Kentucky","Barron County, Wisconsin","Barrow County, Georgia","Barry County, Michigan","Barry County, Missouri","Bartholomew County, Indiana","Barton County, Missouri","Bartow County, Georgia","Bastrop County, Texas","Bates County, Missouri","Bath County, Kentucky","Bath County, Virginia","Baxter County, Arkansas","Bay County, Florida","Bay County, Michigan","Bayfield County, Wisconsin","Beadle County, South Dakota","Beaufort County, North Carolina","Beaufort County, South Carolina","Beauregard Parish, Louisiana","Beaver County, Pennsylvania","Beaverhead County, Montana","Beckham County, Oklahoma","Bedford City and County, Virginia","Bedford County, Pennsyl

In [8]:
features.shape

(2708, 2707)

In [9]:
label.head()

,"Age-Adjusted Incidence Rate - cases per 100,000"
0,76.5
1,67.2
2,37.4
3,38.9
4,37.5


### Train/Test Split

In [10]:
X_train, X_test, y_train, y_test = mlf.train_test_split(features, label, test_size=0.2, random_state=42)

## Run and Evaluate Several Linear Models

In [15]:
import numpy as np
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
#Note: Referenced Sklean documentation

classifiers = [
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]

for item in classifiers:
    print(item)
    clf = item
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('Coefficients: \n', clf.coef_)
    print("Mean squared error: %.2f"% mean_squared_error(y_test, y_pred))
    print('Variance score: %.2f' % r2_score(y_test, y_pred))
    print(" ")
    print(" ")
    print(" ")


BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)
Coefficients: 
 [-4.43417542e-04 -3.76006192e-08  0.00000000e+00 ... -1.89348332e-07
 -3.77635860e-07  0.00000000e+00]
Mean squared error: 256.33
Variance score: 0.19
 
 
 
LassoLars(alpha=1.0, copy_X=True, eps=2.220446049250313e-16,
     fit_intercept=True, fit_path=True, max_iter=500, normalize=True,
     positive=False, precompute='auto', verbose=False)
Coefficients: 
 [0. 0. 0. ... 0. 0. 0.]
Mean squared error: 317.19
Variance score: -0.00
 
 
 
TheilSenRegressor(copy_X=True, fit_intercept=True, max_iter=300,
         max_subpopulation=10000, n_jobs=1, n_subsamples=None,
         random_state=None, tol=0.001, verbose=False)
Coefficients: 
 [-4.32227734e-04 -8.03037084e+00  2.79528390e-14 ... -2.69434878e+01
 -5.01842152e+01  0.00000000e+00]
Mean squared error: 263.58
Variance score: 0.1